In [1]:
import os, random
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers

In [2]:
config = {
    "dataset_path": "images/",
    "validation_ratio": 0.2,
    "image_size": (300, 300),
    "input_shape": (300, 300, 3),
    "batch_size": 32,
    "mode": "train",
    "loss_function": "categorical_crossentropy",
    "optimizer": "adam",
    "epochs": 8,
    "dropout": 0.0,
    "checkpoint_pattern": "checkpoint/brute-force/cp-{epoch:03d}.ckpt",
}

# train/test 데이터셋 나누기

In [3]:
from prep_image.loader import image_loader

# 전처리 과정 정의

In [5]:
from prep_image.preprocess import preprocess

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

# 모델 정의하기

In [5]:
def create_model(config):
    print('system >> creating a model...')
    model = define_model(config)
    mode = config['mode']

    if mode in ["retrain", "test"]:
        print('system >> loading pretrained model...', config['saved_model'])
        model.load_weights(config['saved_model'])
        
    model.compile(loss=config['loss_function'], optimizer=config['optimizer'], metrics=['accuracy'])
    return model

def define_model(config):
    bf = ks.models.Sequential([
        layers.Conv2D(32, (4, 4,), padding="same", activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Conv2D(64, (4, 4,), padding="same", activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Conv2D(128, (4, 4,), padding="same", activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Conv2D(256, (4, 4,), padding="same", activation='relu'),
    ])

    classifier = ks.models.Sequential([
        layers.Flatten(),
        layers.Dropout(config['dropout']),
        layers.Dense(200, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    
    model = ks.models.Sequential([
        normalizer(config['input_shape']),
        incep_rv2,
        classifier
    ])
    return model

# 콜백 작성하기

In [6]:
class CustomCallback(ks.callbacks.Callback):

    def __init__(self):
        self.t_loss = {}
        self.t_acc = {}
        
        self.v_loss = []
        self.v_acc = []
        self.step = 0

    def on_epoch_end(self, epoch, logs):
        print('\n==========\n')
        print('Epoch {} result'.format(epoch))
        
        print('Training Loss:', logs['loss'])
        print('Traininig Accuracy:', logs['accuracy'])
        
        print('Validation Loss:', logs['val_loss'])
        print('Validation Accuracy:', logs['val_accuracy'])
        self.v_loss.append(logs['val_loss'])
        self.v_acc.append(logs['val_accuracy'])
        
    def on_train_batch_end(self, batch, logs=None):
        self.t_acc[self.step] = logs['accuracy']
        self.t_loss[self.step] = logs['loss']
        self.step += 1

    def on_train_end(self, logs):
        import matplotlib.pyplot as plt
        plt.plot(self.t_loss.keys(), self.t_loss.values())
        plt.xlabel('Training Steps')
        plt.ylabel('Training Loss')
        plt.show()
        
recorder = CustomCallback()
checkpoint_callback = ks.callbacks.ModelCheckpoint(
    filepath=config["checkpoint_pattern"], 
    verbose=1,
    monitor='val_accuracy',
    save_weights_only=True,
    save_best_only=True,
    save_freq='epoch'
)

# 학습과 테스트 정의하기

In [7]:
def main(config):
    model = create_model(config)
    print('system >> model summary')
    model.summary()
    
    if config['mode'] in ['train', 'retrain']:
        do_train(model, config)
        do_eval(model, config)
    else:
        do_eval(model, config)

def do_train(model, config):
    shape = (config['batch_size'], *config['image_size'], 3)
    train_ds, valid_ds = image_loader(config['dataset_path'], config['validation_ratio'], 
                                  config['image_size'], config['batch_size'])
    print('system >> identified classes:', *train_ds.class_names)
    
    train_ds = preprocess(train_ds, shape=shape, shuffle=True, augment=True)
    valid_ds = preprocess(valid_ds, shape=shape)
    print('system >> training begins...')
    model.fit(train_ds, 
          validation_data=valid_ds, 
          epochs=config['epochs'],
          callbacks=[recorder, checkpoint_callback])

def do_eval(model, config):
    shape = (config['batch_size'], *config['image_size'], 3)
    valid_ds = image_loader(config['dataset_path'], config['validation_ratio'], 
                                  config['image_size'], config['batch_size'], subset='valid')
    print('system >> identified classes:', *valid_ds.class_names)
    
    valid_ds = preprocess(valid_ds, shape=shape)
    print('system >> evaluation begins...')
    model.evaluate(valid_ds)

def do_test(model, config):
    print('system >> test begins...')

In [40]:
main(config)

system >> creating a model...
system >> loading pretrained model... checkpoint/cp-031-inc.ckpt
system >> model summary
Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 8, 8, 1536)        54336736  
_________________________________________________________________
sequential_25 (Sequential)   (None, 3)                 19661603  
Total params: 73,998,339
Trainable params: 19,661,603
Non-trainable params: 54,336,736
_________________________________________________________________
Found 45000 files belonging to 3 classes.
Using 9000 files for validation.
system >> identified classes: 0food 1interior 2exterior
system >> evaluation begins...
141/141 [==============================] - 51s 323ms/step - loss: 0.1549 - accuracy: 0.9468
